In [1]:
import numpy as np;
import pandas as pd;
import plotly.express as px;


In [2]:
# Global parameters
NO_OF_DAYS = 200
modelParameter = {}
modelParameter["beta"] = 0.49  
modelParameter["tE"] = 5
modelParameter["tI"] = 5
modelParameter["tH"] = 9
modelParameter["mu"] = 0.001

print(modelParameter)

# System variables
ROOT_DIR = "/Users/sreeramh/Documents/Kerala_Covid/SpatioTemporal/gitsrc/SpatioTemporalSEIR"

{'beta': 0.49, 'tE': 5, 'tI': 5, 'tH': 9, 'mu': 0.001}


In [3]:
# Load data
initDataDF = pd.read_csv(ROOT_DIR + "/data/sample_init_data.csv")
#print(initDataDF)

cSparseMatrix = pd.read_csv(ROOT_DIR + "/data/sample_cmatrix.csv")
#print(neighborDF)


In [4]:
## Differential equation
def deriv(y, Ni, contactRatio, modelParameter):
    Si, Ei, Ii, Hi, Ri = y
    dSdt = -modelParameter["beta"] * Si * contactRatio
    dEdt = (modelParameter["beta"] * Si * contactRatio) - (Ei/modelParameter["tE"])
    dIdt = (Ei/modelParameter["tE"]) -  (Ii/modelParameter["tI"])
    dHdt = (Ii/modelParameter["tI"]) - (Hi/modelParameter["tH"])
    dRdt = (Hi/modelParameter["tH"])
    return dSdt, dEdt, dIdt, dHdt, dRdt

In [5]:
def findSpatialContactRatio(name, NDict, yDict, cSparseMatrix):
    # TODO conver this to a matrix operation
    contactRatio = 0;
    Ci = cSparseMatrix[cSparseMatrix['iName']==name]
    for index, Cij in Ci.iterrows():
        jName = Cij['jName']
        cij = Cij['value']
        Ij = yDict[jName][2]
        Nj = NDict[jName]
        contactRatio = contactRatio +  (cij*Ij/Nj)
    return contactRatio;
        

In [6]:
## next count 
def derivNext(y, rates) :
    S,E,I,H, R = y
    dSdt, dEdt, dIdt, dHdt, dRdt = rates
    S1 = S + dSdt
    E1 = E + dEdt
    I1 = I + dIdt
    H1 = H + dHdt
    R1 = R + dRdt
    return S1, E1, I1, H1, R1


In [11]:
yDict = {};
NDict = {};
resultsDF = pd.DataFrame(columns=['name', 'day', 'S', 'E', 'I', 'H', 'R'], );

for index, row in initDataDF.iterrows():
    NDict[row['name']] = row['S'] + row['E'] + row['I'] + row['H'] + row['R']
    yDict[row['name']] = row['S'], row['E'], row['I'], row['H'], row['R'] 
    

for day in range(1,NO_OF_DAYS):
    for index, row in initDataDF.iterrows():
        name = row['name'];
        y = yDict[name]; 
        
        contactRatio = findSpatialContactRatio(name, NDict, yDict, cSparseMatrix);
        #print(name, "contactRatio = ", contactRatio)
        
        rates = deriv(y, NDict[name], contactRatio, modelParameter)
        yNext = derivNext(y, rates) 
        
        yDict[name] = yNext
        #print(name, yNext)
        resultsDF = resultsDF.append({'name': name, 'day': day, 'S': yNext[0], 'E': yNext[1], 'I': yNext[2], 'H': yNext[3], 'R': yNext[4]},ignore_index=True)


In [12]:
resultsDF

,name,day,S,E,I,H,R
0,P1,1,29999.510016,0.489984,0.800000,0.200000,0.000000
1,P1.1,1,14999.998040,0.001960,0.000000,0.000000,0.000000
2,P1.2,1,14999.998040,0.001960,0.000000,0.000000,0.000000
3,P1.1.1,1,7499.999902,0.000098,0.000000,0.000000,0.000000
4,P1.1.2,1,7499.999902,0.000098,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...
1388,P1.2,199,1424.644326,0.505565,0.998153,10.285098,13563.566859
1389,P1.1.1,199,755.404263,1.134608,2.269156,21.109671,6720.082302
1390,P1.1.2,199,755.353069,1.120499,2.241861,20.905057,6720.379514
1391,P1.2.1,199,755.399590,1.131629,2.263226,21.061067,6720.144488


In [13]:
resultsDF.to_csv(ROOT_DIR + "/output/results.csv")

In [14]:
fig = px.scatter(resultsDF, x="day", y="H", color="name")
fig.show()